# Laporan Praktikum Web Scraping

Denis Muhammad Jethro/ 162112133028

### Import Library

In [3]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import scrapy as sc

## Scraping Beautifulsoup

Step:

1. Ambil html dengan requests
2. Masukkan dalam file .html
3. Buka file dan masukkan dalam Beautifulsoup 
4. Find_all tag yang dicari, dalam hal ini *h3*
5. Bersihkan hasilnya dan buat dataframe dari kumpulan data tersebut

In [4]:
url = rq.get("https://unair.ac.id/news")

In [5]:
with open("index.html", 'w', encoding='utf_8') as nulis:
    nulis.write(url.text)

In [6]:
with open("index.html", 'r', encoding='utf_8') as htmll:
    soup = bs(htmll, 'html.parser')

listt = soup.find_all('h3' , class_='elementor-post__title')


In [48]:
data_judul = []
filter = []

for i in listt: 
    splitted = re.split(r'[\n\t\f\v\r]+', i.find('a').get_text())
    
    # Hilangkan enter depan
    for x in splitted[1:]:
        filter.append(x)

# Hilangkan enter belakang
for su in filter[0::2]:
    data_judul.append(su)


### DataFrame Headlines

In [8]:
df = pd.DataFrame(data_judul)
df.columns = ['Judul_Headline']
df.to_csv('../scraping-headlines.csv')

df.head()

,Judul_Headline
0,"Kontribusi Terhadap Keberlanjutan Lingkungan, ..."
1,"Jalani Hidup Sehat, Ini Yang Perlu Disiapkan"
2,5 Mahasiswa UNAIR Ikuti Student Exchange dan K...
3,Waspadai Resistensi Antibiotik Saat Sakit
4,"Kontribusi Terhadap Keberlanjutan Lingkungan, ..."


## Crawling Beautifulsoup

In [9]:
urls = rq.get('https://unair.ac.id/news')

In [10]:
with open('index2.html', 'w', encoding= 'utf_8') as f:
    f.write(urls.text)

with open('index2.html', 'r') as duar:
    soup = bs(urls.text, 'html.parser')


In [12]:
#Dapatkan semua link artikel
all_links = soup.find_all('a', class_='elementor-post__read-more')
news_links = []

for i in all_links: 
    ab = i['href']
    news_links.append(ab)


In [44]:
list_judul = []
list_date = []
#crawler unit (masih bisa dikembangkan)
for i in news_links:
    buang = rq.get(i.lstrip("\'"))

    bakar = bs(buang.text, 'html.parser')
    judul = bakar.find_all('h3', class_='elementor-heading-title')
    date = bakar.find_all('span', class_='elementor-post-info__item--type-date')

    list_judul.append(judul[0].string)
    list_date.append(date[0].string)
    

In [53]:
raw_date = []
filtered_date = []

for so in list_date: 
    splitted = re.split(r'[\n\t\f\v\r]+', so)
    
    # Hilangkan enter depan
    for x in splitted[1:]:
        raw_date.append(x)

# Hilangkan enter belakang
for su in raw_date[0::2]:
    filtered_date.append(su)

### Make and Export DataFrame

In [58]:
df2 = pd.DataFrame({'Judul' : list_judul, 'Date' : filtered_date})
df2.to_csv('../crawling-headlines.csv')

df2.head()

,Judul,Date
0,"Kontribusi Terhadap Keberlanjutan Lingkungan, ...","Oktober 25, 2022"
1,"Jalani Hidup Sehat, Ini Yang Perlu Disiapkan","Oktober 26, 2022"
2,5 Mahasiswa UNAIR Ikuti Student Exchange dan K...,"Oktober 25, 2022"
3,Waspadai Resistensi Antibiotik Saat Sakit,"Oktober 25, 2022"
4,"Kontribusi Terhadap Keberlanjutan Lingkungan, ...","Oktober 25, 2022"
